Давайте добавим пояснения для каждой ячейки кода, чтобы понять процесс.

Эта ячейка кода импортирует необходимые библиотеки: `pandas` для манипулирования данными, `tqdm` для отображения прогресса.

In [ ]:
import pandas as pd
from tqdm import tqdm
import polars as pl

Эта ячейка загружает файл примера отправки (submission) и обучающие данные. Затем она фильтрует обучающие данные, чтобы включить только пользователей, присутствующих в файле submission. Закомментированные строки, похоже, предназначены для потенциальной фильтрации по меньшему набору пользователей (например, топ-100).

In [ ]:
sub = pd.read_csv("sample_submission.csv")

train_data = pd.read_parquet('train_data.pq')
train_data = train_data.loc[train_data['user_id'].isin(sub.user_id.unique())]
# Берем топ 100 юзеров по точности из валидации
#top_100_users = top_100_users_by_accuracy['user_id'].values
#train_data = train_data.loc[train_data['user_id'].isin(top_100_users)]
#sub = sub.loc[sub.user_id.isin(train_data.user_id.unique())]

FileNotFoundError: [Errno 2] No such file or directory: 'sample_submission.csv'

Эта ячейка реализует стратегию рекомендаций, основанную на истории пользователя с учетом затухания по времени и своего рода "когортных" рекомендациях.

- `alpha_user`: Фактор затухания для прошлых взаимодействий.
- `history_days`: Количество прошлых дней для учета истории пользователя.
- Вычисляются взвешенные взаимодействия пользователь-товар на основе давности.
- Создаются словари `yuse` (упорядоченные списки товаров для каждого пользователя) и `yw` (веса товаров для каждого пользователя).
- Определяются топ-товары за последний день (`top_last_week`).
- Создается `dict_top`, который хранит когортные списки для топ-товаров (товары, часто покупаемые вместе с топ-товарами в последний день).

In [ ]:
# Декей истории и когортные соседи (без утечек)
alpha_user = 0.9
history_days = 20

last_day = train_data.date.max()
min_day_user = last_day - (history_days - 1)
tw = train_data.loc[train_data.date >= min_day_user, ['user_id', 'item_id', 'date']].copy()
tw['w'] = (alpha_user ** (last_day - tw['date']))
uw = tw.groupby(['user_id', 'item_id'])['w'].sum().reset_index()
uw = uw.sort_values(['user_id', 'w'], ascending=[True, False])

# упорядоченные пользовательские списки и веса
yuse = {uid: grp['item_id'].tolist() for uid, grp in uw.groupby('user_id')}
yw = {uid: dict(zip(grp['item_id'], grp['w'])) for uid, grp in uw.groupby('user_id')}

# глобальный топ последнего дня
top_last_week = train_data.loc[train_data.date == last_day]['item_id'].value_counts().head(500).index.tolist()

# когортные списки последнего дня для топовых сидов
dict_top = {}
for x in top_last_week[:500]:
    users_x = train_data.loc[train_data.item_id == x, 'user_id'].unique()
    dg = train_data.loc[train_data.user_id.isin(users_x)]
    g = dg.loc[dg.date == last_day]['item_id']
    g = g.value_counts().head(30).index.tolist()
    dict_top[x] = g

NameError: name 'train_data' is not defined

Эта ячейка генерирует предсказания для каждого пользователя в файле submission.

- Для каждого пользователя она начинает с товаров из его взвешенной истории (`yuse`).
- Затем добавляются товары на основе "когортных" оценок, полученных из `dict_top`. Оценка зависит от веса исходного товара из истории пользователя и ранга кандидата в когортном списке.
- Наконец, если список предсказаний все еще меньше 20 товаров, оставшиеся места заполняются товарами из глобального списка `top_last_week`.
- Предсказания затем форматируются в DataFrame для submission и сохраняются как `submission.csv`.

In [ ]:
# Генерация предсказаний и сабмита
prediction = []

for user in tqdm(sub.user_id.unique()):
    user_items = yuse.get(user, [])
    weights = yw.get(user, {})
    max_w = max(weights.values()) if len(weights) > 0 else 1.0

    p = []
    for item in user_items:
        if item not in p:
            p.append(item)
        if len(p) >= 12:
            break

    scores = {}
    for idx, seed in enumerate(user_items[:12]):
        w_seed = (weights.get(seed, 0.0) / max_w) if max_w > 0 else 0.0
        if seed in dict_top:
            for j, nb in enumerate(dict_top[seed]):
                if nb in p:
                    continue
                w_rank = 1.0 / (1.0 + j)
                scores[nb] = scores.get(nb, 0.0) + w_seed * w_rank

    for nb, _ in sorted(scores.items(), key=lambda x: x[1], reverse=True):
        if nb not in p:
            p.append(nb)
        if len(p) >= 20:
            break

    if len(p) < 20:
        for it in top_last_week:
            if it not in p:
                p.append(it)
            if len(p) >= 20:
                break

    prediction.append(p[:20])

# Формирование submission
users = []
items = []
for i, user in tqdm(enumerate(sub.user_id.unique())):
    for item in prediction[i]:
        users.append(user)
        items.append(item)

sub_test = pd.DataFrame({
    'user_id': users,
    'item_id': items,
})

sub_test.to_csv('submission.csv', index=False)

NameError: name 'sub' is not defined

Решения 0.065+

In [ ]:

def exclude_items(lst1, lst2, exclude=True):
    """Remove all items from lst1 that are present in lst2 if exclude=True, otherwise keep only items present in lst2"""
    lst2_set = set(lst2)
    if exclude:
        return [item for item in lst1 if item not in lst2_set]
    else:
        return [item for item in lst1 if item in lst2_set]

sub = pd.read_csv("sample_submission.csv")

train_data = pd.read_parquet('train_data.pq')
train_data = train_data.loc[train_data['user_id'].isin(sub.user_id.unique())]

In [ ]:
use_items = train_data.loc[train_data['date']>=46-10].groupby('user_id')['item_id'].apply(set).to_dict()
top_last = train_data.loc[train_data['date']==46]['item_id'].value_counts().head(20).index.tolist()


top_last_week = train_data.loc[train_data.date == 46]['item_id'].value_counts().head(500).index.tolist()

dict_top = {}

for x in tqdm(top_last_week[:150]):
    users__ = train_data.loc[train_data.item_id==x].user_id.unique()
    dg = train_data.loc[train_data.user_id.isin(users__)]
    g = dg.loc[dg.date == 46]['item_id']
    g = g.value_counts().head(100).index.tolist()
    dict_top[x] = g

In [ ]:
# Генерация предсказаний и сабмита
prediction = []

for user in tqdm(sub.user_id.unique()):
    use_ = []
    if user in use_items:
        use_ = list(use_items[user])

    custom_top = [dict_top[x] for x in use_ if x in dict_top]
    if len(custom_top) != 0:
        custom_top = custom_top[0]

    p = use_ + exclude_items(custom_top, use_)
    p = p + exclude_items(top_last, p)
    prediction.append(p[:20])

users = []
items = []
for i, user in tqdm(enumerate(sub.user_id.unique())):
    for item in prediction[i]:
        users.append(user)
        items.append(item)

sub_test = pd.DataFrame({
    'user_id': users,
    'item_id': items,
})

sub_test.to_csv('submission.csv', index=False)
